In [1]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
from torch.nn.functional import sigmoid

def load_model():
    # Load model and tokenizer
    model_path = '/content/drive/MyDrive/Colab Notebooks/544/project/saved'
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained('roberta-base')

    # Move to GPU if available
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = model.to(device)
    model.eval()

    return model, tokenizer, device

def predict_professor(comment, model, tokenizer, device):
    """
    Predict ratings and tags for a professor's comment
    Returns:
        - difficulty rating (0-5)
        - student rating (0-5)
        - gives_good_feedback (boolean)
        - caring (boolean)
        - respected (boolean)
    """
    # Tokenize input
    inputs = tokenizer(comment, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Get predictions
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

        # First two outputs are ratings (0-5)
        difficulty = logits[0, 0].item()
        student_rating = logits[0, 1].item()

        # Last three outputs are binary tags
        tags = sigmoid(logits[0, 2:]).cpu().numpy()

    return {
        'difficulty_rating': round(difficulty, 2),
        'student_rating': round(student_rating, 2),
        'gives_good_feedback': bool(tags[0] > 0.5),
        'caring': bool(tags[1] > 0.5),
        'respected': bool(tags[2] > 0.5),
        'tag_probabilities': {
            'gives_good_feedback': round(float(tags[0]) * 100, 2),
            'caring': round(float(tags[1]) * 100, 2),
            'respected': round(float(tags[2]) * 100, 2)
        }
    }

# Load model
model, tokenizer, device = load_model()

# Interactive prediction loop
while True:
    # Get user input
    comment = input("\nEnter professor comment (or 'quit' to exit): ")

    if comment.lower() == 'quit':
        break

    # Get predictions
    results = predict_professor(comment, model, tokenizer, device)

    # Print results
    print("\nPredictions:")
    print(f"Difficulty Rating: {results['difficulty_rating']}/5")
    print(f"Student Rating: {results['student_rating']}/5")
    print("\nTags:")
    print(f"Gives Good Feedback: {results['gives_good_feedback']} ({results['tag_probabilities']['gives_good_feedback']}%)")
    print(f"Caring: {results['caring']} ({results['tag_probabilities']['caring']}%)")
    print(f"Respected: {results['respected']} ({results['tag_probabilities']['respected']}%)")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(



Enter professor comment (or 'quit' to exit): this is good

Predictions:
Difficulty Rating: 2.26/5
Student Rating: 4.41/5

Tags:
Gives Good Feedback: False (30.85%)
Caring: False (26.96%)
Respected: False (29.02%)

Enter professor comment (or 'quit' to exit): "Trouble is, if you have different opin- ions on the topics you talk about then she won´t like you. As long as you stick to what she lays out you should be good. She wants very badly to have students she molds, if she can´t mold you she´ll throw you away.

Predictions:
Difficulty Rating: 3.8/5
Student Rating: 2.83/5

Tags:
Gives Good Feedback: False (25.24%)
Caring: False (21.31%)
Respected: False (20.96%)

Enter professor comment (or 'quit' to exit): quit
